In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/post-ontology-label.csv')
df

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')
df

,post,prefLabel,compulsion,obsession,expand1
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."
...,...,...,...,...,...
9991,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...",0,0,"morbid thoughts, Mental image, Homicidal thoug..."
9992,Hi all\nThere are lots of helpful topics on th...,['OCD'],0,0,NaN
9993,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...",0,0,"castration anxiety, performance anxiety, koro,..."
9994,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']",1,0,"terror, panic, panic, moral panic"


In [ ]:
df['post'] = df['post'].fillna('')
df['post'] = df['post'].apply(process_text)
df.head()

,post,prefLabel,compulsion,obsession,expand1
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [ ]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(df, test_size=test_split, stratify=df[['compulsion']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['compulsion']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 7996
Number of rows in validation set: 1000
Number of rows in test set: 1000


In [ ]:
train_x,train_y = train_df['post'], train_df[['compulsion']]
test_x,test_y = test_df['post'], test_df[['compulsion']]
val_x,val_y = val_df['post'], val_df[['compulsion']]

In [ ]:
# Count the number of labels as 0 and 1 for each group
print("Training set:")
print(train_y['compulsion'].value_counts())

print("\nValidation set:")
print(val_y['compulsion'].value_counts())

print("\nTest set:")
print(test_y['compulsion'].value_counts())

Training set:
0    5835
1    2161
Name: compulsion, dtype: int64

Validation set:
0    729
1    271
Name: compulsion, dtype: int64

Test set:
0    730
1    270
Name: compulsion, dtype: int64


#### COMPULSION MODEL

In [11]:

import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
compulsion_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
compulsion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = compulsion_model.fit(
    train_encodings,
    train_labels,
    validation_data=(test_encodings, test_labels),
    epochs=2,
    batch_size=32
)

# Evaluate the model
predictions = compulsion_model.predict(test_encodings)
predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)




predictions = compulsion_model.predict(val_encodings)
predicted_labels = tf.argmax(predictions.logits, axis=1)

from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(val_labels, predicted_labels)
# Calculate accuracy
accuracy = accuracy_score(val_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"conf matrix: {conf_matrix}")





# Parse the classification report to get precision, recall, and F1-score
classification_metrics = classification_rep.split('\n')[2:-5]  # Exclude unnecessary lines
precision, recall, f1_score, _ = map(float, classification_metrics[1].split()[1:])

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
250/250 [==============================] - 11889s 47s/step - loss: 0.2765 - accuracy: 0.8893 - val_loss: 0.1865 - val_accuracy: 0.9370
Epoch 2/2
32/32 [==============================] - 424s 13s/step
Accuracy: 0.932
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95       730
           1       0.95      0.79      0.86       270

    accuracy                           0.93      1000
   macro avg       0.94      0.89      0.91      1000
weighted avg       0.93      0.93      0.93      1000

32/32 [==============================] - 426s 13s/step
Accuracy: 0.948
conf matrix: [[725   4]
 [ 48 223]]
Precision: 0.95
Recall: 0.79
F1-Score: 0.86
